* 대한소아마취학회 apm 저널 중 APM002-01-08.md file 깔끔하게 후처리 해서 RAG 구현.
    - Table 구조화하여 마크다운 형식 대체
    - \n 제거 및 문단 구조 정리
    - Figure 맥락 추출해서 요약 및 삽입

## Markdown 후처리

In [9]:
pip install openai

Note: you may need to restart the kernel to use updated packages.


In [8]:
input_path = r"C:\Users\user\OneDrive\바탕 화면\BOAZ\2025_분석_ADV session\챗봇 프로젝트\QAData\Apm002-01-08_new.md"
output_path = input_path.replace(".md", "_restructured.md")

# 마크다운 불러오기 → 구조화 변환 → 저장
original_md = load_markdown_file(input_path)
updated_md = restructure_all_tables(original_md)
save_markdown_file(output_path, updated_md)

print(f"✅ 모든 테이블 변환 완료! 저장 위치: {output_path}")


❌ 변환할 테이블이 없습니다.
✅ 모든 테이블 변환 완료! 저장 위치: C:\Users\user\OneDrive\바탕 화면\BOAZ\2025_분석_ADV session\챗봇 프로젝트\QAData\Apm002-01-08_new_restructured.md


In [10]:
import re
from pathlib import Path
from openai import OpenAI
from dotenv import load_dotenv

# ✅ 환경변수 로드 (OPENAI_API_KEY 포함된 .env 파일 필요)
load_dotenv()
client = OpenAI()

# ✅ 파일 경로 설정
input_path = r"C:\Users\user\OneDrive\바탕 화면\BOAZ\2025_분석_ADV session\챗봇 프로젝트\QAData\Apm002-01-08_new.md"
output_path = input_path.replace(".md", "_restructured.md")

# ✅ 마크다운 로딩
def load_markdown_file(path):
    return Path(path).read_text(encoding="utf-8")

# ✅ 마크다운 저장
def save_markdown_file(path, content):
    Path(path).write_text(content, encoding="utf-8")

# ✅ GPT 구조화 요청 프롬프트 생성
def create_structured_prompt(table_md: str) -> str:
    return f"""
다음은 마크다운 형식의 표입니다. 각 행(row)을 LLM이 이해하기 쉬운 구조로 아래처럼 재구성해주세요.

요청 형식:
- [Column1]: 값
  - [Column2]: 값
  - ...

표:
{table_md}

요청:
- 표의 각 행을 bullet-point로 정리해주세요.
- 헤더 명은 그대로 사용해주세요.
"""

# ✅ GPT-4o 호출
def gpt_convert_markdown_table(table_md: str) -> str:
    prompt = create_structured_prompt(table_md)
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )
    return response.choices[0].message.content

# ✅ Table 1만 처리
def restructure_table_1(md_text: str) -> str:
    # \n 이 이스케이프 되어있는 경우를 실제 줄바꿈으로 변환
    md_text = md_text.replace("\\n", "\n")
    lines = md_text.splitlines()

    # Table 1 헤딩 탐지
    heading = "Table 1. Congenital Anomalies and Its Anesthetic Implications"
    heading_idx = next((i for i, line in enumerate(lines) if heading in line), None)
    if heading_idx is None:
        print("❌ 'Table 1.' 헤딩을 찾을 수 없습니다.")
        return md_text

    # 마크다운 테이블 시작 줄 찾기
    table_start = None
    for i in range(heading_idx, len(lines)):
        if "No. of patients" in lines[i] and "|" in lines[i]:
            table_start = i
            break

    if table_start is None:
        print("❌ 표 시작을 찾을 수 없습니다.")
        return md_text

    # 마크다운 테이블 끝 줄 찾기
    table_end = None
    for j in range(table_start + 1, len(lines)):
        if not lines[j].strip().startswith("|"):
            table_end = j
            break

    if table_end is None:
        table_end = len(lines)

    # 표 줄 추출
    table_lines = lines[table_start:table_end]
    table_md = "\n".join(table_lines)

    # GPT로 구조화 요청
    structured = gpt_convert_markdown_table(table_md)

    # 원문 갱신
    new_lines = lines[:table_start] + [structured] + lines[table_end:]
    return "\n".join(new_lines)

# ✅ 실행
original_md = load_markdown_file(input_path)
updated_md = restructure_table_1(original_md)
save_markdown_file(output_path, updated_md)
print(f"✅ Table 1 변환 완료! 저장 위치: {output_path}")


✅ Table 1 변환 완료! 저장 위치: C:\Users\user\OneDrive\바탕 화면\BOAZ\2025_분석_ADV session\챗봇 프로젝트\QAData\Apm002-01-08_new_restructured.md


In [11]:
from pathlib import Path

# 저장된 파일 다시 불러오기
output_path = r"C:\Users\user\OneDrive\바탕 화면\BOAZ\2025_분석_ADV session\챗봇 프로젝트\QAData\Apm002-01-08_new_restructured.md"
updated_text = Path(output_path).read_text(encoding="utf-8")

# 1. Table 1 이후 50줄 미리보기
lines = updated_text.splitlines()
preview = []

found_table = False
for i, line in enumerate(lines):
    if "Table 1." in line:
        preview = lines[i:i+50]
        found_table = True
        break

if not found_table:
    print("❌ 'Table 1.' 문구를 찾을 수 없습니다.")
else:
    print("✅ 'Table 1.' 이후의 내용 (미리보기):\n")
    for l in preview:
        print(l)


✅ 'Table 1.' 이후의 내용 (미리보기):

Table 1. Congenital Anomalies and Its Anesthetic Implications

- Disease: Down syndrome
  - No. of patients: 17
  - Associated disease: ASD, VSD, PDA, hypotonia, hyperthyroidism, Graves' disease
  - Anesthetic consideration/reasons for difficult intubation: Atlantoaxial instability, macroglossia, short neck

- Disease: Marfan syndrome
  - No. of patients: 8
  - Associated disease: MVP, AR, MR, ulcerative colitis, hypothyroidism
  - Anesthetic consideration/reasons for difficult intubation: Cervical spinal abnormality

- Disease: Klippel-Feil syndrome
  - No. of patients: 5
  - Associated disease: Block vertebrae cervical spine (2-3), spina bifida cervical spine (7), microcephaly
  - Anesthetic consideration/reasons for difficult intubation: Vertebral anomalies (cervical vertebral fusion, scoliosis), unstable short neck, cleft palate

- Disease: Pierre-Robin syndrome
  - No. of patients: 3
  - Associated disease: Corpus callosum agenesis
  - Anesthetic consi

## document 구성

In [34]:
import re
from typing import TypedDict, Annotated
from langchain_core.documents import Document
from langgraph.graph import StateGraph
from langchain_core.runnables import Runnable
from langchain_core.runnables.base import RunnableLambda
from langchain.vectorstores import Pinecone
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
import os

In [47]:
#  마크다운 파일 로드
markdown_path = r"C:\Users\user\OneDrive\바탕 화면\BOAZ\2025_분석_ADV session\챗봇 프로젝트\QAData\Apm002-01-08_new_restructured.md"

with open(markdown_path, encoding="utf-8") as f:
    markdown_text = f.read()

In [48]:
# 커스텀 문단 분리 함수 (테이블 블록은 [ ... ] 내부로 처리)
def split_paragraphs_with_block_tag(text: str):
    paragraphs = []
    buffer = []
    inside_block = False

    for line in text.splitlines():
        if line.strip().startswith("["):
            inside_block = True
            buffer = [line]
            continue
        elif line.strip().endswith("]"):
            buffer.append(line)
            paragraphs.append("\n".join(buffer).strip())
            buffer = []
            inside_block = False
            continue

        if inside_block:
            buffer.append(line)
        else:
            if line.strip() == "":
                if buffer:
                    paragraphs.append("\n".join(buffer).strip())
                    buffer = []
            else:
                buffer.append(line)

    if buffer:
        paragraphs.append("\n".join(buffer).strip())

    return [p for p in paragraphs if p.strip()]



In [49]:
# 문단 추출
paragraphs = split_paragraphs_with_block_tag(markdown_text)

In [50]:
# 메타데이터 정의
base_metadata = {
    "document_id": "Apm002-01-08",
    "title": "An Experience of Anesthesia of Congenital Anomaly with Airway Involvement - 46 case report -",
    "authors": [
        "Tae Joong Yoo", "Sang Seok Lee", "Byung Hoon Yoo",
        "Seung Hoon Woo", "Yeun Hee Lim", "Woo Yong Lee"
    ],
    "year": 2007,
    "keywords": ["airway", "craniofacial abnormality", "pediatric"],
    "summary": "As the improved medical techniques and environmental changes have increased the frequency of general anesthesia for uncommon congenital anomalies. The airway management for a patient with congenital anomaly gives significant challenges to the anesthesiologist. The purpose of this report is to review the authors’ experience with airway management and ventilatory support during the perioperative period in children with congenital anomalies with airway involvement, and to summarize anesthetic implications associated with particular congenital anomalies by literature review. Total 46 cases of general anesthesia for operation of congenital anomalies were performed from January 2000 to August 2005 in our hospital. Most common congenital anomaly is a Down syndrome (17 cases, 37%), and most common cause of surgery is the orthopedic surgery for correction of deformed spine (18 cases, 41%). Direct laryngoscopy was successfully used to establish an airway in 35 (76.1%) cases, whereas 5 (11%) cases required the use of fiberoptic bronchoscopy to establish an airway before surgery."
}

In [51]:
# LangChain Document 리스트 생성
documents = []
for idx, para in enumerate(paragraphs):
    metadata = {**base_metadata, "paragraph_id": idx}
    documents.append(Document(page_content=para, metadata=metadata))

In [54]:
documents

[Document(metadata={'document_id': 'Apm002-01-08', 'title': 'An Experience of Anesthesia of Congenital Anomaly with Airway Involvement - 46 case report -', 'authors': ['Tae Joong Yoo', 'Sang Seok Lee', 'Byung Hoon Yoo', 'Seung Hoon Woo', 'Yeun Hee Lim', 'Woo Yong Lee'], 'year': 2007, 'keywords': ['airway', 'craniofacial abnormality', 'pediatric'], 'summary': 'As the improved medical techniques and environmental changes have increased the frequency of general anesthesia for uncommon congenital anomalies. The airway management for a patient with congenital anomaly gives significant challenges to the anesthesiologist. The purpose of this report is to review the authors’ experience with airway management and ventilatory support during the perioperative period in children with congenital anomalies with airway involvement, and to summarize anesthetic implications associated with particular congenital anomalies by literature review. Total 46 cases of general anesthesia for operation of congen

## Embedding & Pinecon DB 적재

In [57]:
from langchain.vectorstores import Pinecone
from langchain.embeddings import OpenAIEmbeddings
from pinecone import Pinecone, ServerlessSpec
import time
from tqdm import tqdm
from uuid import uuid4
from langchain_pinecone import PineconeVectorStore

pinecone_api = ''

# Initialize a Pinecone client with your API key
openai_api_key = os.environ["OPENAI_API_KEY"]
pinecone_api_key = os.environ["PINECONE_API_KEY"]
pc = Pinecone(api_key=pinecone_api_key)

index_name = "quickstart"

# OpenAI 임베딩 생성
embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002",
    api_key=openai_api_key
)

# 임베딩 모델 차원 확인 (예: OpenAI의 'text-embedding-ada-002'는 1536차원)
EMBEDDING_DIMENSION = 1536  # OpenAI 임베딩 모델 차원

# 인덱스가 이미 존재하는 경우 생성하지 않고 사용
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name, 
        dimension=EMBEDDING_DIMENSION,  # 임베딩의 차원을 설정 (OpenAI 임베딩의 차원)
        metric='cosine',
        spec=ServerlessSpec(cloud='aws', region='us-east-1')
    )
    # 인덱스가 준비될 때까지 대기
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

# 인덱스 가져오기
index = pc.Index(index_name)

# PineconeVectorStore를 사용하여 벡터 스토어 생성
vectorstore = PineconeVectorStore(index=index, embedding=embeddings, text_key="page_content")

# 문서에 UUID 할당
uuids = [str(uuid4()) for _ in range(len(documents))]

for doc, uid in tqdm(zip(documents, uuids), total=len(documents), desc="Pinecone에 문서 추가 중"):
    vectorstore.add_documents(documents=[doc], ids=[uid])

print(" 문서 저장이 완료되었습니다.")

C:\Users\user\AppData\Local\Temp\ipykernel_92396\2760434527.py:19: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(
Pinecone에 문서 추가 중: 100%|██████████| 17/17 [00:23<00:00,  1.39s/it]

 문서 저장이 완료되었습니다.


In [58]:
# 적재된 Pinecone DB에서 데이터 불러오기
import os
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv

# langsmith 추적
load_dotenv()

# API키 불러오기
open_api_key = os.environ['OPENAI_API_KEY']
pinecone_api = os.environ['PINECONE_API_KEY']

# OpenAI 임베딩 생성
embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002",  # OpenAI의 임베딩 모델 사용
    api_key=open_api_key)

# 인덱스 가져오기
pc = Pinecone(api_key=pinecone_api)
index_name = "quickstart"
index = pc.Index(index_name)

# PineconeVectorStore를 사용하여 벡터 스토어 생성
vectorstore = PineconeVectorStore(index=index, 
                                  embedding=embeddings, 
                                  text_key="page_content")

## LangGraph

In [89]:
# 1. 환경 변수 불러오기 및 기본 설정
import os
from dotenv import load_dotenv
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
INDEX_NAME = "quickstart"

In [90]:
# 2. LangGraph 노드 정의 (langgraph_node.py 역할)
from typing import Annotated, List, TypedDict
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import AIMessage
from langgraph.graph import StateGraph, START, END
from langchain_community.document_transformers import LongContextReorder

class ChatbotState(TypedDict):
    question: Annotated[str, "Question"]
    documents: Annotated[List, "Context"]
    chatbot: Annotated[str, "Answer"]
    messages: Annotated[List, add_messages]

memory = MemorySaver()

def build_graph(model, prompt, retriever):
    reorder = LongContextReorder()

    def retrieve_document(state: ChatbotState):
        question = state["question"]
        docs = retriever.invoke(question)
        ordered = reorder.transform_documents(docs)
        return ChatbotState(documents=ordered)

    def llm_answer(state: ChatbotState):
        question = state["question"]
        docs = state["documents"]
        docs = "\n\n".join(doc.page_content.strip() for doc in docs)
        formatted = prompt.format(context=docs, question=question)
        response = model.invoke(formatted)
        response_text = response.content if isinstance(response, AIMessage) else str(response)
        return ChatbotState(chatbot=response_text,
                            messages=[("user", question), ("assistant", response_text)])

    builder = StateGraph(ChatbotState)
    builder.add_node("docs", retrieve_document)
    builder.add_node("llm_answer", llm_answer)
    builder.add_edge(START, "docs")
    builder.add_edge("docs", "llm_answer")
    builder.add_edge("llm_answer", END)

    return builder.compile(checkpointer=memory)

In [91]:
# 3. 모델 설정 함수 (model.py 역할)
from langchain_community.chat_models import ChatOllama
from langchain_openai import ChatOpenAI
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_community.llms import HuggingFacePipeline
import torch

def create_llama_model():
    model_id = "meta-llama/Llama-3.2-1B-Instruct"
    model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype="auto", device_map="auto")
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=1024, do_sample=True,
                   temperature=0.3, pad_token_id=tokenizer.eos_token_id, repetition_penalty=1.1)
    return HuggingFacePipeline(pipeline=pipe)

In [92]:
# 4. 프롬프트 설정 함수 (prompt.py 역할)
from langchain_core.prompts import PromptTemplate

def build_Llama_prompt():
    return PromptTemplate.from_template(
        """
<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
You are an assistant specialized in question-answering tasks based on medical research papers.
Use the following pieces of retrieved context to answer the question. If you don't know the answer, simply say that you don't know.
Answer in Korean.
<|start_header_id|>user<|end_header_id|>
Context: {context}
Question: {question}
<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
Answer:
""")

In [94]:
# 5. retriever 설정 함수 (retriever.py 역할)
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings

def create_retriever(openai_key, pinecone_key, index_name):
    embeddings = OpenAIEmbeddings(model="text-embedding-ada-002", api_key=openai_key)
    pc = Pinecone(api_key=pinecone_key)
    index = pc.Index(index_name)
    vectorstore = PineconeVectorStore(index=index, embedding=embeddings, text_key="page_content")
    base_retriever = vectorstore.as_retriever(search_kwargs={"k": 10})
    reranker = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-v2-m3")
    compressor = CrossEncoderReranker(model=reranker, top_n=5)
    return ContextualCompressionRetriever(base_retriever=base_retriever, base_compressor=compressor)



In [95]:
# 6. 실행 코드 (main.py 역할)
from langchain_core.runnables import RunnableConfig

def run_chatbot(question: str, thread_id: str = "1"):
    model = create_llama_model()
    prompt = build_Llama_prompt()
    retriever = create_retriever(OPENAI_API_KEY, PINECONE_API_KEY, INDEX_NAME)
    graph = build_graph(model, prompt, retriever)
    config = RunnableConfig(configurable={"thread_id": thread_id})
    input_state = {"question": question, "messages": []}

    for event in graph.stream(input_state, config=config):
        for value in event.values():
            if 'documents' in value:
                for idx, doc in enumerate(value['documents']):
                    print(f"\n📄 {idx+1}번째 문서\n📌 문서 ID: {doc.metadata.get('document_id')}\n📑 문단 번호: {doc.metadata.get('paragraph_id')}")
                    print(doc.page_content)
            if 'chatbot' in value:
                full_text = value["chatbot"]
                print("\n💬 답변:")
                print(full_text)

In [ ]:
# 7. 인터랙션 시작
question = input("질문 입력: ")
run_chatbot(question)